In [3]:
# -*- coding:utf-8 -*-
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
# from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
from hmmlearn import hmm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# 15.2　特征提取

## 15.2.1　标准化

In [8]:
df = pd.read_csv("../Data/creditcard.csv")
df


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [9]:
df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,normAmount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.244964
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,-0.342475
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1.160686
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.140534
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,-0.073403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0,-0.350151
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0,-0.254117
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0,-0.081839
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0,-0.313249


In [10]:
df = df.drop(['Time', 'Amount'], axis=1)
df


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0,-0.350151
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,0,-0.254117
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,0,-0.081839
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,0,-0.313249


In [11]:
y=df['Class']
print(y)

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64


In [12]:
features = df.drop(['Class'], axis=1).columns

In [13]:
x=df[features]
x

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.350151
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,-0.254117
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,-0.081839
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.313249


In [14]:

def get_feature():
    df = pd.read_csv("../Data/creditcard.csv")
    df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
    df = df.drop(['Time', 'Amount'], axis=1)

    y=df['Class']
    features = df.drop(['Class'], axis=1).columns
    x=df[features]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test  = get_feature()

## 15.2.2　标准化和降采样

In [16]:
def get_feature_undersampling():
    df = pd.read_csv("../Data/creditcard.csv")
    df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
    df = df.drop(['Time', 'Amount'], axis=1)

    number_fraud=len(df[df.Class==1])
    #print(number_fraud
    fraud_index=np.array(df[df.Class==1].index)
    #print(fraud_index

    normal_index=df[df.Class==0].index
    random_choice_index=np.random.choice(normal_index,size=number_fraud,replace=False)

    x_index=np.concatenate([fraud_index,random_choice_index])
    df = df.drop(['Class'], axis=1)
    x=df.iloc[x_index,:]
    #print(x
    y=[1]*number_fraud+[0]*number_fraud


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_feature_undersampling()

In [24]:
pd.DataFrame(y_train).value_counts()

0    297
1    293
dtype: int64

## 15.2.3　标准化和过采样

In [18]:

def get_feature_undersampling_2():
    df = pd.read_csv("../Data/creditcard.csv")
    df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
    df = df.drop(['Time', 'Amount'], axis=1)

    y = df['Class']
    features = df.drop(['Class'], axis=1).columns
    x = df[features]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

    print("raw data")
    print(pd.value_counts(y_train))

    number_fraud=len(y_train[y_train==1])
    print(number_fraud)
    fraud_index=np.array(y_train[y_train==1].index)
    print(fraud_index)

    normal_index=y_train[y_train==0].index
    random_choice_index=np.random.choice(normal_index,size=number_fraud,replace=False)

    x_index=np.concatenate([fraud_index,random_choice_index])
    x_train_1=x.iloc[x_index,:]

    y_train_1=[1]*number_fraud+[0]*number_fraud

    print("Undersampling data")
    print(pd.value_counts(y_train_1))
    return x_train_1, x_test, y_train_1, y_test
x_train_1, x_test, y_train_1, y_test = get_feature_undersampling_2()

raw data
0    170581
1       303
Name: Class, dtype: int64
303
[184379 261473 154286 275992  10690 116404  68320  30473 248296  27627
 234705 149869 102444  10497 151462 213116 250761  79525  41395  15476
 128479 191690 147605 106679 102782   6971 124087 251866 199896 152223
   9179   6719 135718  83417  83297  74496  30100 141259  40085  61787
  69498 150684   6472 111690 231978 251891  43160 141258 154697 150644
  81186 143728  58422  44556 154371 192382  42700  42674  75511  74507
 150647   6338 239499 124115 189878 249963 123141  30314 154676 268375
  73784 125342 154719 226877  70589 151006  50537  88258 245556 149600
 144754 215953  96994 150677  42887  27749  43204 263877  15166 154670
 107637 204064 118308  27738 222133  59539  69980  77387 255556  42528
  81609   6609 154234   8617 150679  30442 191267 189587 191544 150697
  10204 143336 151807 212644 150687 157585 239501 191359 276071 143333
 189701 272521  26802 123201  55401  93486   6641  84543 150678  29687
  42741 262826

In [23]:
pd.DataFrame(y_train_1).value_counts()

0    303
1    303
dtype: int64

In [27]:
def get_feature_upsampling():
    df = pd.read_csv("../Data/creditcard.csv")
    df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
    df = df.drop(['Time', 'Amount'], axis=1)

    y = df['Class']
    features = df.drop(['Class'], axis=1).columns
    x = df[features]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

    print("raw data")
    print(pd.value_counts(y_train))

    os = SMOTE(random_state=0)
    x_train_1,y_train_1=os.fit_resample(x_train,y_train)
    print("Smote data")
    print(pd.value_counts(y_train_1))

    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_feature_upsampling()

raw data
0    170593
1       291
Name: Class, dtype: int64
Smote data
0    170593
1    170593
Name: Class, dtype: int64


# 15.3　模型训练与验证

In [29]:
def do_metrics(y_test,y_pred):
    print("metrics.accuracy_score:")
    print(metrics.accuracy_score(y_test, y_pred))
    print("metrics.confusion_matrix:")
    print(metrics.confusion_matrix(y_test, y_pred))
    print("metrics.precision_score:")
    print(metrics.precision_score(y_test, y_pred))
    print("metrics.recall_score:")
    print(metrics.recall_score(y_test, y_pred))
    print("metrics.f1_score:")
    print(metrics.f1_score(y_test,y_pred))

def do_xgboost(x_train, x_test, y_train, y_test):
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    do_metrics(y_test, y_pred)

In [30]:
def do_mlp(x_train, x_test, y_train, y_test):
    #mlp
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes=(5, 2),
                        random_state=1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    do_metrics(y_test,y_pred)

In [31]:

def do_nb(x_train, x_test, y_train, y_test):
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    do_metrics(y_test,y_pred)

In [32]:
def run_1():
    x_train, x_test, y_train, y_test=get_feature()
    do_xgboost(x_train, x_test, y_train, y_test)
    do_mlp(x_train, x_test, y_train, y_test)
    do_nb(x_train, x_test, y_train, y_test)
run_1()


metrics.accuracy_score:
0.9996225520746469
metrics.confusion_matrix:
[[113719      8]
 [    35    161]]
metrics.precision_score:
0.9526627218934911
metrics.recall_score:
0.8214285714285714
metrics.f1_score:
0.8821917808219177
metrics.accuracy_score:
0.9994206613238766
metrics.confusion_matrix:
[[113709     18]
 [    48    148]]
metrics.precision_score:
0.891566265060241
metrics.recall_score:
0.7551020408163265
metrics.f1_score:
0.8176795580110497
metrics.accuracy_score:
0.9788892497564144
metrics.confusion_matrix:
[[111354   2373]
 [    32    164]]
metrics.precision_score:
0.06464327946393378
metrics.recall_score:
0.8367346938775511
metrics.f1_score:
0.12001463593121112


In [33]:

def run_2():
    x_train, x_test, y_train, y_test=get_feature_undersampling()
    print("XGBoost")
    do_xgboost(x_train, x_test, y_train, y_test)
    print("mlp")
    do_mlp(x_train, x_test, y_train, y_test)
    print("nb")
    do_nb(x_train, x_test, y_train, y_test)
run_2()

XGBoost
metrics.accuracy_score:
0.9365482233502538
metrics.confusion_matrix:
[[183   7]
 [ 18 186]]
metrics.precision_score:
0.9637305699481865
metrics.recall_score:
0.9117647058823529
metrics.f1_score:
0.9370277078085643
mlp
metrics.accuracy_score:
0.9137055837563451
metrics.confusion_matrix:
[[177  13]
 [ 21 183]]
metrics.precision_score:
0.9336734693877551
metrics.recall_score:
0.8970588235294118
metrics.f1_score:
0.915
nb
metrics.accuracy_score:
0.8984771573604061
metrics.confusion_matrix:
[[185   5]
 [ 35 169]]
metrics.precision_score:
0.9712643678160919
metrics.recall_score:
0.8284313725490197
metrics.f1_score:
0.8941798941798942


In [34]:
def run_3():
    x_train, x_test, y_train, y_test=get_feature_upsampling()
    print("XGBoost")
    do_xgboost(x_train, x_test, y_train, y_test)
    print("mlp")
    do_mlp(x_train, x_test, y_train, y_test)
    print("nb")
    do_nb(x_train, x_test, y_train, y_test)
run_3()

raw data
0    170599
1       285
Name: Class, dtype: int64
Smote data
0    170599
1    170599
Name: Class, dtype: int64
XGBoost
metrics.accuracy_score:
0.9996049963571886
metrics.confusion_matrix:
[[113707      9]
 [    36    171]]
metrics.precision_score:
0.95
metrics.recall_score:
0.8260869565217391
metrics.f1_score:
0.8837209302325583
mlp


d:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


metrics.accuracy_score:
0.9994206613238766
metrics.confusion_matrix:
[[113701     15]
 [    51    156]]
metrics.precision_score:
0.9122807017543859
metrics.recall_score:
0.7536231884057971
metrics.f1_score:
0.8253968253968254
nb
metrics.accuracy_score:
0.9774321252073769
metrics.confusion_matrix:
[[111176   2540]
 [    31    176]]
metrics.precision_score:
0.06480117820324006
metrics.recall_score:
0.8502415458937198
metrics.f1_score:
0.12042422169004449
